In [3]:
import os
import numpy as np

meta_file = "C:\\users\\joris\\Documents\\eScience_data\\data\\Similarity_matrix_AllPositive\\metadata_AllInchikeys.csv"
sim_file = "C:\\users\\joris\\Documents\\eScience_data\\data\\Similarity_matrix_AllPositive\\similarities_AllInchikeys_daylight2048_jaccard.npy"
os.path.isfile(meta_file), os.path.isfile(sim_file)

(True, True)

In [29]:
with open(meta_file, 'r') as inf:
    header = inf.readline()
    lines = inf.readlines()

add_lines = {}
keep_inds = []
for i, line in enumerate(lines):
    line = line.strip().split(',')
    inchi = line[1]
    uniq_inchi = inchi.partition("-")[0]
    if uniq_inchi not in add_lines:
            new_line = ','.join([uniq_inchi]+line[2:])
            keep_inds.append((i, uniq_inchi))
            add_lines[uniq_inchi] = new_line
    else:
            add_lines[uniq_inchi] += ';'+ line[-1]

In [30]:
len(keep_inds), len(add_lines), len(lines)

(12846, 12846, 14459)

In [26]:
inds, inchi14s = zip(*keep_inds)
sim_map = np.lib.format.open_memmap(sim_file, dtype="float64", mode="r")
row_slice = np.take(sim_map, inds, 0)
final_slice = np.take(row_slice, inds, 1)
final_slice, final_slice.shape

(memmap([[1.        , 0.09930716, 0.23619371, ..., 0.21175523, 0.33166144,
          0.18229623],
         [0.09930716, 1.        , 0.09296782, ..., 0.09761635, 0.11690141,
          0.09446694],
         [0.23619371, 0.09296782, 1.        , ..., 0.26397248, 0.30480949,
          0.41290323],
         ...,
         [0.21175523, 0.09761635, 0.26397248, ..., 1.        , 0.33022388,
          0.25946445],
         [0.33166144, 0.11690141, 0.30480949, ..., 0.33022388, 1.        ,
          0.2511041 ],
         [0.18229623, 0.09446694, 0.41290323, ..., 0.25946445, 0.2511041 ,
          1.        ]]),
 (12846, 12846))

In [27]:
output_folder = "C:\\users\\joris\\Documents\\eScience_data\\data\\Similarity_matrix_AllPositive\\"
sim_output = os.path.join(output_folder, "similarities_AllInchikeys14_daylight2048_jaccard.npy")
metadata_output = os.path.join(output_folder, "metadata_AllInchikeys14.csv")

In [35]:
print(sim_output)

C:\users\joris\Documents\eScience_data\data\Similarity_matrix_AllPositive\similarities_AllInchikeys14_daylight2048_jaccard.npy


In [28]:
final_slice = np.array(final_slice)
np.save(sim_output, final_slice)
del(final_slice)

In [33]:
with open(metadata_output, 'w') as outf:
    outf.write(header)
    for i, (old_ind, inchi14) in enumerate(keep_inds):
        line_no_i = add_lines[inchi14]
        new_line = f"{i},{line_no_i}\n"
        outf.write(new_line)